# Data Staging
### Extract, Transform, Load (ETL)

**Create Data Frame for Facility Locations in Ontario**


In [254]:
import pandas as pd

Extract Facility Location

In [255]:
# read file containing information about recreation centre locations and types
# facility location dataframe
df = pd.read_csv('./ODRSF_v1.0.csv', encoding='cp1252')

C:\Users\Arthur\AppData\Local\Temp\ipykernel_11700\371614087.py:3: DtypeWarning: Columns (17,18) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('./ODRSF_v1.0.csv', encoding='cp1252')


Transform Facility Dataframe

In [256]:
# Remove unneeded columns
df = df.drop(columns = ['Index', 'Facility_Name', 'Source_Facility_Type', 'Provider', 'City', 'Source_Format_Address', 'CSD_UID', 'PR_UID', 'Latitude', 'Longitude'])

# Remove unit since almost all values are null
df = df.drop(columns = ['Unit'])

In [257]:
#Replace empty values of street_no with -1
df.loc[df['Street_No'] == '..', 'Street_No'] = '-1'

# Replace rest of street strings with an empty value
df.loc[df['Street_Name'] == '..', 'Street_Name'] = ''
df.loc[df['Street_Type'] == '..', 'Street_Type'] = ''
df.loc[df['Street_Direction'] == '..', 'Street_Direction'] = ''
df.loc[df['Street_Direction'] == '..', 'Street_Direction'] = ''
df.loc[df['Postal_Code'] == '..', 'Postal_Code'] = ''

In [258]:
#convert street_no to integer
df['Street_No'] = df['Street_No'].astype(int)

In [259]:
# Captialize all province/territory column
df['Prov_Terr'] = df['Prov_Terr'].str.upper()

In [260]:
# remove all non ontario provinces
df = df[df['Prov_Terr'] == 'ON']

In [261]:
# replace sudbury areas with just sudbury

# Remove unnecessary information after comma in geo_name so processing can be easier
df['CSD_Name'] = df['CSD_Name'].str.split(',').str[0]

In [262]:
#unique_cities = df['City'].unique()
unique_cities = df['CSD_Name'].unique()

In [263]:
column_rename_dict = {
    'CSD_Name': 'City',
    'ODRSF_facility_type': 'FacilityType',
}

df = df.rename(columns=column_rename_dict)

**Create Dataframe for City Demographics in Ontario**

In [264]:
demographic_df = pd.read_csv('./Ontario_Demographics.csv', encoding='cp1252')

In [265]:
demographic_df = demographic_df.drop(columns = ['CENSUS_YEAR', 'DGUID', 'ALT_GEO_CODE', 'GEO_LEVEL', 'TNR_SF', 'TNR_LF', 'DATA_QUALITY_FLAG','SYMBOL', 'CHARACTERISTIC_ID', 'CHARACTERISTIC_NOTE', 'C2_COUNT_MEN+', 'C3_COUNT_WOMEN+', 'C10_RATE_TOTAL', 'C11_RATE_MEN+', 'C12_RATE_WOMEN+', 'SYMBOL.1','SYMBOL.2','SYMBOL.3','SYMBOL.4','SYMBOL.5'])

In [266]:
# Remove unnecessary information after comma in geo_name so processing can be easier
demographic_df['GEO_NAME'] = demographic_df['GEO_NAME'].str.split(',').str[0]

# only keep rows where the city is both in the demographic dataframe and the facility location dataframe
demographic_df = demographic_df[demographic_df['GEO_NAME'].isin(unique_cities)]

In [267]:
#get starting and ending index of rows that contain age information about the city
age_indexes = demographic_df[demographic_df['CHARACTERISTIC_NAME'].str.contains('Total - Age groups of the population - 100% data|100 years and over')].index

#get starting and ending index of rows that contain ethnicity information about the city
ethnicity_indexes = demographic_df[demographic_df['CHARACTERISTIC_NAME'].str.contains('Total visible minority population|Total - Ethnic or cultural origin for the population in private households - 25% sample data')].index

In [268]:
selected_age_dfs = []
selected_ethnicity_dfs = []

# store only data in dataframe from start and end indexes
for i in range(0, len(age_indexes), 2):
    age_df = demographic_df[age_indexes[i] + 1: age_indexes[i + 1] + 1]
    ethnicity_df = demographic_df[ethnicity_indexes[i] + 1: ethnicity_indexes[i + 1]]
    selected_age_dfs.append(age_df)
    selected_ethnicity_dfs.append(ethnicity_df)

#resulting dataframes 
result_age_df = pd.concat(selected_age_dfs, ignore_index=True)
result_ethnicity_df = pd.concat(selected_ethnicity_dfs, ignore_index=True)

# since some 'cities' have multiple occurences(i.e, different regions, we will need to keep each of their unique values.)
# To make the ETL process easier, we want to make it so each city in their respective data occurs only once
prev_city = None
for index, row in result_age_df.iterrows():
    # Check if current city is different from previous city
    if row['GEO_NAME'] == prev_city:
        # Drop the row if the city is the same as the previous one
        result_age_df.at[index, 'GEO_NAME'] = ''
    else:
        # Update previous city
        prev_city = row['GEO_NAME']

# special cases where these CSDs are right after each other so the for loop above does not work
result_age_df.at[4875, 'GEO_NAME'] = 'Nipissing'
result_age_df.at[5200, 'GEO_NAME'] = 'Parry Sound'

prev_city = None
for index, row in result_ethnicity_df.iterrows():
    # Check if current city is different from previous city
    if row['GEO_NAME'] == prev_city:
        # Drop the row if the city is the same as the previous one
        result_ethnicity_df.at[index, 'GEO_NAME'] = ''
    else:
        # Update previous city
        prev_city = row['GEO_NAME']    

# special cases where these CSDs are right after each other so the for loop above does not work
result_ethnicity_df.at[2535, 'GEO_NAME'] = 'Nipissing'
result_ethnicity_df.at[2703, 'GEO_NAME'] = 'Parry Sound'

In [269]:
# set empty values/null to 0
result_age_df.fillna({'C1_COUNT_TOTAL': 0}, inplace=True)
# check for null
result_age_df['C1_COUNT_TOTAL'].isnull().sum()

0

In [270]:
# set empty values/null to 0
result_ethnicity_df.fillna({'C1_COUNT_TOTAL': 0}, inplace=True)
# check for null
result_ethnicity_df['C1_COUNT_TOTAL'].isnull().sum()

0

In [271]:
# get all cities from the dataframe(cities are the same from the age dataframe and tehnicity dataframe)
cities = result_age_df[result_age_df['GEO_NAME'] != '']['GEO_NAME']

Transpose Age Dataframe

In [272]:
# Unique age ranges - will be used as the new column in the new dataframe
ages = result_age_df['CHARACTERISTIC_NAME'].unique()

age_df = pd.DataFrame(columns=ages)

# add new column for city
age_df.insert(loc=0, column='City', value='')

# remove whitespace from columns
age_df = age_df.rename(columns=lambda x: x.strip())

In [273]:
# Populate first column cities with all city variables
age_df['City'] = cities
age_count = result_age_df['C1_COUNT_TOTAL'].tolist()

for i in range(len(cities)):
    if i == 0:
        age_df.iloc[i, 1:] = age_count[:25]
    else:
        age_df.iloc[i, 1:] = age_count[i * 25 : (i + 1) * 25]
        
for column in age_df.columns[1:]:
    age_df[column] = age_df[column].astype(int)

# Sum duplicate rows since we consider regions of a City as the same City, and sort data by City
age_df = age_df.groupby('City').sum().reset_index()

In [274]:
# redundant data that can be dropped
age_df = age_df.drop(columns = ['0 to 14 years', '15 to 64 years','65 years and over'])   

Transpose Ethnicity Dataframe

In [275]:
# Unique ethnicities
ethnicities = result_ethnicity_df['CHARACTERISTIC_NAME'].unique()

ethnicity_df = pd.DataFrame(columns=ethnicities)

ethnicity_df.insert(loc=0, column='City', value='')

# remove whitespace from columns
ethnicity_df = ethnicity_df.rename(columns=lambda x: x.strip())

In [276]:
# Populate first column with all cities
ethnicity_df['City'] = cities
ethnicity_count = result_ethnicity_df['C1_COUNT_TOTAL'].tolist()

# Populate column data
k = 0
for i in range(len(cities)):
    if i == 0:
        ethnicity_df.iloc[i, 1:] = ethnicity_count[:13]
    else:
        ethnicity_df.iloc[i, 1:] = ethnicity_count[i * 13 : (i + 1) * 13]

#change type from float to int
for column in ethnicity_df.columns[1:]:
    ethnicity_df[column] = ethnicity_df[column].astype(int)

# Sum duplicate rows since we consider regions of a City as the same City, and sort data by City
ethnicity_df = ethnicity_df.groupby('City').sum().reset_index()

In [277]:
unique_facilities = df['FacilityType'].unique()

In [278]:
# output all dataframes
age_df.to_csv('age_df.csv', index=False)
ethnicity_df.to_csv('eth_df.csv', index=False)
df.to_csv('facility_df.csv', index=False)

**Generating Measure/Fact Table**

In [282]:
fact_df = pd.DataFrame()
fact_df['City'] = age_df['City']
fact_df['Facility Count'] = None
fact_df['Highest Population Age'] = None
fact_df['Highest Population Ethnicity'] = None
fact_df = fact_df.sort_values(by='City').reset_index(drop=True)


print(len(fact_df))

for i in range(len(age_df)):
    fact_df.loc[i, 'Highest Population Age'] = (age_df.iloc[i, 1:]).idxmax()
    fact_df.loc[i, 'Highest Population Ethnicity'] = (ethnicity_df.iloc[i, 1:]).idxmax()
    fact_df.loc[i, 'Facility Count'] = (df['City'] == fact_df.loc[i, 'City']).sum()

242


,City,Facility Count,Highest Population Age,Highest Population Ethnicity
0,Addington Highlands,57,60 to 64 years,Not a visible minority
1,Adjala-Tosorontio,20,55 to 59 years,Not a visible minority
2,Ajax,32,55 to 59 years,Not a visible minority
3,Alderville First Nation,2,55 to 59 years,Not a visible minority
4,Alfred and Plantagenet,65,55 to 59 years,Not a visible minority
...,...,...,...,...
237,Whitchurch-Stouffville,201,40 to 44 years,Not a visible minority
238,Wilmot,264,35 to 39 years,Not a visible minority
239,Windsor,786,25 to 29 years,Not a visible minority
240,Woodstock,87,30 to 34 years,Not a visible minority


In [280]:
fact_df.to_csv('fact_df.csv', index=False)